# Library importing

In [1]:
import pandas as pd
import joblib
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 296.7/296.8 MB 271.4 MB/s eta 0:00:01

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 296.7/296.8 MB 271.4 MB/s eta 0:00:01

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.8/296.8 MB 38.5 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/1.8 MB ? eta -:--:--

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.0 MB/s eta 0:00:00


  Attempting uninstall: pip
    Found existing installation: pip 25.0.1


    Uninstalling pip-25.0.1:


      Successfully uninstalled pip-25.0.1


# data pre-proccessing

## data reading

In [2]:
chunks = pd.read_csv("/kaggle/input/depi-dataset/data.csv", chunksize=5_000_000)
final_data = pd.concat(chunks, ignore_index=True)

In [3]:
del chunks

In [4]:
preprocessor=joblib.load("/kaggle/input/depi-dataset/pre-proccess.pkl")

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator OrdinalEncoder from version 1.7.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.7.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator Pipeline from version 1.7.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info 

## Data spliting

In [5]:
y=final_data['sales']
x=final_data.drop('sales',axis = 1)

# Model

In [6]:
del final_data
import gc
gc.collect()

0

In [7]:
categorical_cols = ['store_id', 'event_name_1', 'event_name_2', 'event_type_1', 'event_type_2', 'state_id', 'item_category', 'item_subcategory']
numeric_cols=['wm_yr_wk', 'wday', 'snap', 'year', 'month', 'day', 'sell_price', 'lag_1','price_flag','item_number']
for col in categorical_cols:
    if x[col].dtype == "object":
        x[col] = x[col].astype("category")
for col in numeric_cols:
    x[col] = pd.to_numeric(x[col], downcast="float")
del categorical_cols
del numeric_cols
gc.collect()

0

In [8]:
del col

In [9]:
gc.collect()

0

In [10]:
x=preprocessor.fit_transform(x)

In [28]:
model = XGBRegressor(
    objective='count:poisson',
    booster='gbtree',
    device='cuda',
    tree_method='hist',
    random_state=42,
    eval_metric='mae',
    learning_rate=0.05,
    max_depth=10,
    min_child_weight=5,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=6,
    reg_alpha=2,
    n_estimators=3000,
    early_stopping_rounds=40,
    gamma=0.1
)

In [12]:
x_temp , x_test , y_temp , y_test = train_test_split(x , y , test_size = 0.20 , random_state = 42)
x_train , x_val , y_train , y_val = train_test_split(x_temp , y_temp , test_size = 0.15 , random_state = 42)

In [13]:
del x
del y

In [20]:
gc.collect()

0

In [29]:
model.fit(x_train,y_train,eval_set = [(x_val , y_val)])

[0]	validation_0-mae:1.51684
[1]	validation_0-mae:1.50327
[2]	validation_0-mae:1.49003
[3]	validation_0-mae:1.47801
[4]	validation_0-mae:1.47012
[5]	validation_0-mae:1.45881
[6]	validation_0-mae:1.44783
[7]	validation_0-mae:1.43725
[8]	validation_0-mae:1.42701
[9]	validation_0-mae:1.42210
[10]	validation_0-mae:1.41180
[11]	validation_0-mae:1.40173
[12]	validation_0-mae:1.39169
[13]	validation_0-mae:1.38195
[14]	validation_0-mae:1.37496
[15]	validation_0-mae:1.36557
[16]	validation_0-mae:1.35600
[17]	validation_0-mae:1.34666
[18]	validation_0-mae:1.33757
[19]	validation_0-mae:1.33119
[20]	validation_0-mae:1.32308
[21]	validation_0-mae:1.31464
[22]	validation_0-mae:1.30642
[23]	validation_0-mae:1.30105
[24]	validation_0-mae:1.29290
[25]	validation_0-mae:1.28726
[26]	validation_0-mae:1.27938
[27]	validation_0-mae:1.27153
[28]	validation_0-mae:1.26613
[29]	validation_0-mae:1.26090
[30]	validation_0-mae:1.25349
[31]	validation_0-mae:1.24669
[32]	validation_0-mae:1.24174
[33]	validation_0-ma

XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device='cuda', early_stopping_rounds=40,
             enable_categorical=False, eval_metric='mae', feature_types=None,
             gamma=0.1, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=5, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=3000, n_jobs=None,
             num_parallel_tree=None, objective='count:poisson', ...)

[79]	validation_0-mae:0.68075


[80]	validation_0-mae:0.68003


[81]	validation_0-mae:0.67925


...

[1996]	validation_0-mae:0.56038


[1997]	validation_0-mae:0.56037


[1998]	validation_0-mae:0.56035


[1999]	validation_0-mae:0.56034


,objective,'reg:squarederror'
,base_score,None
,booster,'gbtree'
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,'cuda'
,early_stopping_rounds,40
,enable_categorical,False
,eval_metric,'mae'


In [30]:
model.score(x_train,y_train) , model.score(x_test,y_test)

(0.7698343603496113, 0.7271533161861972)

In [32]:
import numpy as np
y_predict=model.predict(x_test)
y_pred_rounded = np.round(y_predict).astype(int)
y_test_rounded = np.round(y_test).astype(int)
for i in range (0,25):
    print(y_pred_rounded[i], y_test_rounded.iloc[i])

4 2
1 1
1 1
1 1
0 0
0 0
1 0
0 0
2 2
0 0
0 0
0 0
7 15
1 0
1 1
2 4
0 0
0 0
1 0
1 3
0 0
0 0
0 0
0 0
0 1


In [34]:
from sklearn.metrics import mean_absolute_error, r2_score

train_mae = mean_absolute_error(y_train, model.predict(x_train))
test_mae  = mean_absolute_error(y_test,  model.predict(x_test))
train_r2  = r2_score(y_train, model.predict(x_train))
test_r2   = r2_score(y_test,  model.predict(x_test))

print(f"MAE train: {train_mae:.4f}, test: {test_mae:.4f}")
print(f"R2  train: {train_r2:.4f}, test: {test_r2:.4f}")


MAE train: 0.8008, test: 0.8189
R2  train: 0.7698, test: 0.7272
